In [5]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.utils.np_utils import to_categorical  

# (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))

# centre the data
def centre_data(train, validation, test):
    
    # calculate the means for each attribute of the training data
    column_means = np.mean(train, axis=0) 
    
    # centre training data by subtracting training data attribute means
    for i in range(len(train)):
        train[i] = train[i] - column_means
    
    # centre testing data by subtracting training data attribute means
    for x in range(len(test)):
        test[x] = test[x] - column_means
    
    for x in range(len(validation)):
        validation[x] = validation[x] - column_means
        
    return train, validation, test

# apply PCA on the data 
def PCA(variance_target, training_data, validation_data, testing_data):

    U, sigma, Vt = np.linalg.svd(training_data, full_matrices=False)
    
    sum_square_singular = np.sum(sigma**2)
    
    ratios = sigma**2/sum_square_singular
    
                
    n_components = 0
    explained_variance = 0
    
    # determine how many principle components must be retained to maintain the target level of explained variance
    for i in range(len(ratios)):
        if explained_variance >= variance_target:
            break
        else: 
            n_components += 1
            explained_variance += ratios[i]
    
    return training_data.dot(Vt.T[:, :n_components]), testing_data.dot(Vt.T[:, :n_components]), validation_data.dot(Vt.T[:, :n_components])

    
# training_labels = to_categorical(training_labels, 20)
# testing_labels = to_categorical(testing_labels, 20)

# # Transform images from (32,32,3) to 3072-dimensional vectors (32*32*3)

# training_data = np.reshape(training_data,(50000, 3072))
# testing_data = np.reshape(testing_data,(10000, 3072))

# training_data = training_data.astype('float32')
# testing_data = testing_data.astype('float32')

# training_data, testing_data = centre_data(training_data, testing_data)

# training_data, testing_data = PCA(0.8, training_data, testing_data)

# training_labels = to_categorical(training_labels, 20)
# testing_labels = to_categorical(testing_labels, 20)

from keras.datasets import cifar100
import numpy as np

def load_in_dataset_and_preprocess(explained_variance):
    (training_data, training_labels), (testing_data, testing_labels) = (cifar100.load_data("coarse"))

    # reshape the data 
    training_data = training_data.reshape(50000, 3072)
    testing_data = testing_data.reshape(10000, 3072)

    # preprocess data
    validation_data = training_data[49000:, :].astype(np.float)
    validation_labels = np.squeeze(training_labels[49000:, :])
    training_data = training_data[:49000, :].astype(np.float)
    training_labels = np.squeeze(training_labels[:49000, :])
    testing_labels = np.squeeze(testing_labels)
    testing_data = testing_data.astype(np.float)

    # Centre data
    training_data, validation_data, testing_data = centre_data(training_data, validation_data, testing_data)

    # Apply PCA
    training_data, testing_data, validation_data = PCA(explained_variance, training_data, validation_data, testing_data)
    
    return training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels

training_data, training_labels, testing_data, testing_labels, validation_data, validation_labels = load_in_dataset_and_preprocess(0.8)


Epoch 1/15
1225/1225 - 3s - loss: nan - accuracy: 0.0496 - val_loss: nan - val_accuracy: 0.0526
Epoch 2/15
1225/1225 - 4s - loss: nan - accuracy: 0.0496 - val_loss: nan - val_accuracy: 0.0526
Epoch 3/15
1225/1225 - 3s - loss: nan - accuracy: 0.0496 - val_loss: nan - val_accuracy: 0.0526
Epoch 4/15
1225/1225 - 3s - loss: nan - accuracy: 0.0496 - val_loss: nan - val_accuracy: 0.0526
Epoch 5/15
1225/1225 - 4s - loss: nan - accuracy: 0.0496 - val_loss: nan - val_accuracy: 0.0526
Epoch 6/15
1225/1225 - 3s - loss: nan - accuracy: 0.0496 - val_loss: nan - val_accuracy: 0.0526
Epoch 7/15
1225/1225 - 3s - loss: nan - accuracy: 0.0496 - val_loss: nan - val_accuracy: 0.0526
Epoch 8/15
1225/1225 - 4s - loss: nan - accuracy: 0.0496 - val_loss: nan - val_accuracy: 0.0526
Epoch 9/15
1225/1225 - 3s - loss: nan - accuracy: 0.0496 - val_loss: nan - val_accuracy: 0.0526
Epoch 10/15
1225/1225 - 3s - loss: nan - accuracy: 0.0496 - val_loss: nan - val_accuracy: 0.0526
Epoch 11/15
1225/1225 - 4s - loss: nan 

NameError: name 'plt' is not defined

In [ ]:
# Normalization of pixel values (to [0-1] range)

training_labels = to_categorical(training_labels, 20)
testing_labels = to_categorical(testing_labels, 20)

# training_data /= 255
# testing_data /= 255



model = Sequential()

model.add(Dense(256, activation='elu', input_dim=training_data.shape[1]))

model.add(Dense(256, activation='elu'))

model.add(Dense(20, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(training_data, training_labels, epochs=15, batch_size=32, verbose=2, validation_split=0.2)

def plotLosses(history):  
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    
plotLosses(history)

score = model.evaluate(testing_data, testing_labels, batch_size=128, verbose=0)

print(model.metrics_names)
print(score)